Fill these in first

In [ ]:
import os

# Fill In
os.environ['GOOGLE_CLOUD_PROJECT'] = ""

# Generator Setup

In [ ]:

google_cloud_project = os.environ.get("GOOGLE_CLOUD_PROJECT")
# Location where you put your two files in setup 
image_with_people = f"gs://{google_cloud_project}-gcs/entrance with people.jpg"
image_empty = f"gs://{google_cloud_project}-gcs/entrance_empty.jpg" 
user_file = f"gs://{google_cloud_project}-gcs/members.txt"

## Pip installs & Imports

In [ ]:
%pip install google-cloud-storage
%pip install --upgrade google-cloud-pubsub

In [ ]:
import time
import uuid
import random
from google.cloud import storage
from urllib.parse import urlparse
from copy import deepcopy
import os
from google.cloud import pubsub_v1
import json

## Members Lists

In [ ]:
# Helper function to split apart the GCS URI
def decode_gcs_url(url):
    # Read the URI and parse it
    p = urlparse(url)
    bucket = p.netloc
    file_path = p.path[0:].split('/', 1)
    # Return the relevant objects (bucket, path to object)
    return bucket, file_path[1]

# We can't use the image load from local file since it expects a local path
# We use a GCS URL and get the bytes of the image
def read_file(object_path):
    # Parse the path
    bucket, file_path = decode_gcs_url(object_path)
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket)
    blob = bucket.blob(file_path)
    # Return the object as bytes
    return blob.download_as_text()

In [ ]:


class member():
    def __init__(self,first_name,last_name,parking_benefits,tier):
        self.first_name = first_name
        self.last_name = last_name
        self.parking_benefits = parking_benefits
        self.tier = tier

    def to_dict(self):
        return {"first_name":self.first_name, "last_name":self.last_name, "parking_benefits":self.parking_benefits, "tier":self.tier}

class member_list():
    def __init__(self):
        self.members = {}

    def load(self):
        lines = read_file(user_file)
        for line in lines.split('\n')[1:]:
            member_id,first_name,last_name,parking_benefits,tier = line.split('|')
            self.members[member_id] = member(first_name,last_name,parking_benefits,tier)

    def get_random_member(self):
        return random.choice(list(self.members.keys()))

In [ ]:
new_list = member_list()
new_list.load()
for k,v in new_list.members.items():
    print  (k,v.to_dict())

for i in range(10):
    print(new_list.get_random_member())

## Parking

In [ ]:
def gen_parking_log ():
    transaction_id = str(uuid.uuid4())
    member_flag = random.choice([True, False])
    member_id = None
    entry_time = int(time.time())
    if member_flag:
        member_id = new_list.get_random_member()

    return {"transaction_id":transaction_id, "member_id": member_id, "entry_time":entry_time}



In [ ]:
for i in range(10):
    print(gen_parking_log())

## Area of interest

In [ ]:
def area_check ():
    transaction_id = str(uuid.uuid4())
    area = random.choice(["pos","entrance"])
    return {"transaction_id":transaction_id, "area": area}


In [ ]:
for i in range(10):
    print(area_check())

## Line is busy

In [ ]:
def gen_line_status ():
    is_busy = random.choice([True, False])
    area = random.choice(["pos","entrance"])
    if is_busy:
        return {"area":area, "image":image_with_people}
    else:
        return {"area":area, "image":image_empty}

In [ ]:
for i in range(10):
    print(gen_line_status())

## Check In

In [ ]:
def gen_checkin ():
    time_start = int(time.time())
    check_in_list = deepcopy(new_list.members)
    check_in_list = list(check_in_list.keys())
    return_check_ins = []
    while len(check_in_list) > 0:
        member_id = None
        transaction_id = str(uuid.uuid4())
        member_flag = random.choice([True, False])
        if member_flag:
            member_id = random.choice(check_in_list)
            check_in_list.remove(member_id)
        else:
            member_id = None
        check_in_time = time_start + random.randint(1,1000)
        check_in = {"transaction_id":transaction_id,"check_in_time":check_in_time,"member_id": member_id}
        return_check_ins.append(check_in)


    return return_check_ins



In [ ]:
check_ins = gen_checkin()
for check_in in check_ins:
    print(check_in)

## PubSub Write

In [ ]:
pubsub_topics = {
    'parking' : {
        'beam24-workshop-parking-input-topic':
        'beam24-workshop-parking-input-sub'},
    'checkin' : {
        'beam24-workshop-checkin-input-topic':
        'beam24-workshop-checkin-input-sub'},
    'area' : {
        'beam24-workshop-area-input-topic':
        'beam24-workshop-area-input-sub'},
    'parking_output' : {
        'beam24-workshop-parking-output-topic':
        'beam24-workshop-parking-output-sub'},
    'discount_output' : {
        'beam24-workshop-discount-output-topic':
        'beam24-workshop-discount-output-sub'},
    'inventory_output' : {
        'beam24-workshop-inventory-output-topic':
        'beam24-workshop-inventory-output-sub'},
    'line_status': {
        'beam24-workshop-line-input-topic':
        'beam24-workshop-line-input-sub'}}

for area in pubsub_topics.keys():
    for topic, sub in pubsub_topics[area].items():
        os.environ['current_topic'] = topic
        os.environ['current_sub'] = sub
        !gcloud pubsub topics create $current_topic
        !gcloud pubsub subscriptions create $current_sub --topic projects/$GOOGLE_CLOUD_PROJECT/topics/$current_topic


In [ ]:
def pubsub_write(messages, topic_id):
    publisher = pubsub_v1.PublisherClient()
    # The `topic_path` method creates a fully qualified identifier
    # in the form `projects/{project_id}/topics/{topic_id}`
    topic_path = publisher.topic_path(google_cloud_project, topic_id)

    for message in messages:
        # Data must be a bytestring
        message = json.dumps(message)
        data = message.encode("utf-8")
        # When you publish a message, the client returns a future.
        future = publisher.publish(topic_path, data)
        print(future.result())


# Run Generator

In [ ]:
for i in range(100):
    messages = []
    if i%25 == 0:
        pubsub_write([gen_line_status()], "beam24-workshop-line-input-topic")
    if i%5 == 0:
        pubsub_write([area_check()], "beam24-workshop-area-input-topic")
    if i%2 == 0:
        pubsub_write([gen_parking_log()], "beam24-workshop-parking-input-topic")
    if i == 50:
        pubsub_write(gen_checkin(), "beam24-workshop-checkin-input-topic")
    time.sleep(0.25)

